In [1]:
# importing all necessary libraries
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
path = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/RL training data/nn_training_data_first_half.csv"
training_data_first_half = pd.read_csv(path)
path = "/content/drive/MyDrive/Monarch/RL/Tic Tac Toe/RL training data/nn_training_data_second_half.csv"
training_data_second_half = pd.read_csv(path)
training_data = pd.concat([training_data_first_half, training_data_second_half])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
print(len(training_data))
print(training_data.head())


6540829
   Unnamed: 0  0  1  2  3  4  5  6  7  8
0           0  0  0  0  0  0  0  0  0  0
1           1  0  0  0  1  0  0  0  0  0
2           2  2  0  0  1  0  0  0  0  0
3           3  2  0  0  1  0  0  1  0  0
4           4  2  0  0  1  0  0  1  2  0


In [6]:
episode_rewards = []
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Softmax(dim=-1),
        )

    def forward(self, x):
        return self.fc(x)


In [7]:
def compute_discounted_rewards(rewards, gamma=0.99):
    discounted_rewards = []
    R = 0
    for r in reversed(rewards):
        R = r + gamma * R
        discounted_rewards.insert(0, R)
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-5)
    return discounted_rewards
def train(env, policy, optimizer, episodes=1000):
  for episode in range(episodes):
      state = env.reset()
      log_probs = []
      rewards = []
      done = False

      while not done:
          state = torch.FloatTensor(state).unsqueeze(0)
          probs = policy(state)
          m = Categorical(probs)
          action = m.sample()
          state, reward, done, _ = env.step(action.item())

          log_probs.append(m.log_prob(action))
          rewards.append(reward)
          # Inside the train function, after an episode ends:

          if done:
              episode_rewards.append(sum(rewards))
              discounted_rewards = compute_discounted_rewards(rewards)
              policy_loss = []
              for log_prob, Gt in zip(log_probs, discounted_rewards):
                  policy_loss.append(-log_prob * Gt)
              optimizer.zero_grad()
              policy_loss = torch.cat(policy_loss).sum()
              policy_loss.backward()
              optimizer.step()

              if episode % 50 == 0:
                  print(f"Episode {episode}, Total Reward: {sum(rewards)}")
              break

In [ ]:
env = gym.make('CartPole-v1')
policy = PolicyNetwork()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

train(env, policy, optimizer)